In [17]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

import os
root_folder = os.getcwd()
weaving_dir = '../../../'

In [18]:
from scipy.sparse.linalg import spsolve
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "morphing_data", "new_kitty_1.obj"))

In [19]:
linkage_vx = []
linkage_lines = []
with open(os.path.join(root_folder, "morphing_data", "new_kitty_1_line.obj"), 'r') as linkage:
    content = linkage.readlines()
    for line in content:
        if 'v ' in line:
            point = np.array([float(x) for x in line.split(' ')[1:]])
            linkage_vx.append(point)
        else:
            linkage_lines.append(line)

## Compute the barycentric coordinates of vertices of the linkage 

In [20]:
simplex_barycoords = []
for pt in linkage_vx:
    dis, closest_idx, closest_pt = igl.point_mesh_squared_distance(pt, v, f)
    _, _, barycoords = igl.point_simplex_squared_distance(pt, v, f, closest_idx)
    simplex_barycoords.append([closest_idx, barycoords])

In [21]:
np.save('morphing_data/linkage_to_surface_barycoords.npy', np.array(simplex_barycoords))

### Modified Mean Curvature Flow

In [22]:
l = igl.cotmatrix(v, f)
n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
input_doublearea = sum(igl.doublearea(v, f)) 
igl.doublearea(v, f)
vs = [v]
cs = [c]
for i in range(20):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    s = (m - 0.01 * l)
    b = m.dot(v)
    v = spsolve(s, m.dot(v))
    scale = input_doublearea / sum(igl.doublearea(v, f)) 
    v *= np.sqrt(scale)
    n = igl.per_vertex_normals(v, f)*0.5+0.5
    c = np.linalg.norm(n, axis=1)
    vs.append(v)
    cs.append(c)
    igl.write_obj('kitty_level_{}.obj'.format(i), v, f)

p = plot(v, f, c, shading={"wireframe": False}, return_plot=True)
@interact(level=(0, 20))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0058845…

interactive(children=(IntSlider(value=0, description='level', max=20), Output()), _dom_classes=('widget-intera…

In [23]:
for level in range(100)[-1:]:
    v, f = igl.read_triangle_mesh(os.path.join(weaving_dir, "normalized_objs", "surface_models", "kitty_level_{}.obj".format(level)))

    with open(os.path.join(root_folder, "morphing_data", "linkage", "kitty_level_{}.obj".format(level)), 'w') as new_linkage:
        for [closest_idx, barycoords] in simplex_barycoords:
            vxs = f[closest_idx]
            point = np.sum(np.array([v[vxs[i]] * barycoords[i] for i in range(3)]), axis = 0)
            new_linkage.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
        for line in linkage_lines:
            new_linkage.write(line)